# Lens modeling with lenstronomy

Author: Nishuti \
Acknowledgement: Nahid.

In [1]:
# import of standard python libraries
import h5py
import joblib
import corner
import matplotlib.pyplot as plt
import numpy as np
from lenstronomy.Data.coord_transforms import Coordinates
from lenstronomy.Plots.model_plot import ModelPlot
from lenstronomy.Util import mask_util, util
from lenstronomy.Util.param_util import ellipticity2phi_q
from lenstronomy.Workflow.fitting_sequence import FittingSequence
from lenstronomy.Analysis.light_profile import LightProfileAnalysis
from lenstronomy.LightModel.light_model import LightModel

%matplotlib inline

d:\Study\Astronomy_Astrophysics\env_astro\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


ImportError: cannot import name 'isiterable' from 'astropy.cosmology.utils' (d:\Study\Astronomy_Astrophysics\env_astro\Lib\site-packages\astropy\cosmology\utils.py)

### Load imaging data

In [ ]:
with h5py.File(r"DESIJ0618+5018_F140W.h5", "r") as f:
    kwargs_data = {}
    for key in f:
        kwargs_data[key] = f[key][()]

image_data = kwargs_data["image_data"]

In [ ]:
with h5py.File(r"psf_F140W.h5", "r") as f:
    kwargs_psf = {}
    for key in f:
        kwargs_psf[key] = f[key][()]
kwargs_psf

psf_map = kwargs_psf["kernel_point_source"]

### Visualizing the data

In [ ]:
image_data = kwargs_data["image_data"]
psf_map = kwargs_psf["kernel_point_source"]

In [ ]:
kwargs_psf = {
    "psf_type": "PIXEL",
    "kernel_point_source": psf_map,
}

In [ ]:
plt.matshow(np.log10(image_data), origin="lower", cmap="cubehelix")

### likelihood mask using lenstronomy functions

In [ ]:
ra_at_xy_0 = kwargs_data["ra_at_xy_0"]
dec_at_xy_0 = kwargs_data["dec_at_xy_0"]
transform_pix2angle = kwargs_data["transform_pix2angle"]

coords = Coordinates(transform_pix2angle, ra_at_xy_0, dec_at_xy_0)
num_pix = len(kwargs_data["image_data"])

x_coords, y_coords = coords.coordinate_grid(num_pix, num_pix)

print(x_coords, y_coords)

r = 5.3  # arcseconds
lens_center_ra = 0
lens_center_dec = 0

mask_outer = mask_util.mask_center_2d(
    lens_center_ra + 0.2,
    lens_center_dec,
    r,
    util.image2array(x_coords),
    util.image2array(y_coords),
)


# For the galaxy at 1 o clock

mask_ext_1 = mask_util.mask_ellipse(
    util.image2array(x_coords),
    util.image2array(y_coords),
    lens_center_ra - 1.1,
    lens_center_dec + 3.05,
    0.40,
    0.73,
    -7.5,
)

# For the galaxy at 1 o clock (bottom)

mask_ext_2 = mask_util.mask_ellipse(
    util.image2array(x_coords),
    util.image2array(y_coords),
    lens_center_ra - 1.4,
    lens_center_dec + 2.72,
    0.30,
    0.53,
    -6.5,
)

# For the galaxy at 4 o clock

mask_ext_3 = mask_util.mask_ellipse(
    util.image2array(x_coords),
    util.image2array(y_coords),
    lens_center_ra - 2.9,
    lens_center_dec - 2.3,
    0.29,
    0.29,
    0,
)

# For the galaxy at 8 o clock

mask_ext_4 = mask_util.mask_ellipse(
    util.image2array(x_coords),
    util.image2array(y_coords),
    lens_center_ra + 3.37,
    lens_center_dec - 1.1,
    0.26,
    0.26,
    0,
)

# For the galaxy at Center

mask_ext_5 = mask_util.mask_ellipse(
    util.image2array(x_coords),
    util.image2array(y_coords),
    lens_center_ra,
    lens_center_dec,
    0.25,
    0.25,
    0,
)


mask = (
    (1 - mask_outer)
    * (1 - mask_ext_1)
    * (1 - mask_ext_2)
    * (1 - mask_ext_3)
    * (1 - mask_ext_4)
    * (1 - mask_ext_5)
)


mask[mask >= 1] = 1
mask[mask < 0] = 0

mask = mask.reshape(num_pix, num_pix)

plt.matshow(mask, origin="lower", cmap="cubehelix")
plt.show()

# mask image data
masked_image_data = np.multiply(image_data, mask)

plt.matshow(np.log10(masked_image_data), origin="lower", cmap="cubehelix")
plt.minorticks_on()
plt.show()

### Compiling the lens model

In [ ]:
lens_model_list = ["EPL", "SHEAR", "SIE"]
source_model_list = ["SERSIC_ELLIPSE", "SHAPELETS"]
lens_light_model_list = ["SERSIC_ELLIPSE", "SERSIC_ELLIPSE", "SERSIC_ELLIPSE"]

### Mass model of the lens galaxy

In [ ]:
# lens galaxy's mass model
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []

# setting EPL parameters
fixed_lens.append({"gamma": 2})
kwargs_lens_init.append({"theta_E": 2.2872,
                         "gamma": 2.,
                         "e1": -0.1690,
                         "e2": 0.0376,
                         "center_x": -0.4456,
                         "center_y": 0.1274,})

kwargs_lens_sigma.append({"theta_E": 0.2,
                          "gamma": 0.1,
                          "e1": 0.05,
                          "e2": 0.05,
                          "center_x": 0.5,
                          "center_y": 0.5,})

kwargs_lower_lens.append({"theta_E": 2.0,
                          "gamma": 1.5,
                          "e1": -0.5,
                          "e2": -0.5,
                          "center_x": -10,
                          "center_y": -10,})

kwargs_upper_lens.append({"theta_E": 10.0,
                          "gamma": 3.0,
                          "e1": 0.5,
                          "e2": 0.5,
                          "center_x": 10,
                          "center_y": 10,})

# Setting SHEAR Parameters
fixed_lens.append({"ra_0": 0, "dec_0": 0})
kwargs_lens_init.append({"gamma1": -0.0795,
                         "gamma2": 0.0374,
                         "ra_0": 0,
                         "dec_0": 0})

kwargs_lens_sigma.append({"gamma1": 0.1,
                          "gamma2": 0.1,
                          "ra_0": 0,
                          "dec_0": 0})

kwargs_lower_lens.append({"gamma1": -0.3,
                          "gamma2": -0.3,
                          "ra_0": -100,
                          "dec_0": -100})

kwargs_upper_lens.append({"gamma1": 0.3,
                          "gamma2": 0.3,
                          "ra_0": 100,
                          "dec_0": 100})

# #SIE
satellite_center_x = - 2.7175
satellite_center_y = 1.2716
satellite_centroid_bound = 0.4

fixed_lens.append({})
kwargs_lens_init.append({'theta_E': 0.04,
                         'e1': 0.0056,
                         'e2': 0.0018,
                         'center_x': satellite_center_x,
                         'center_y': satellite_center_y})
kwargs_lens_sigma.append({'theta_E': .2,
                          'e1': 0.05,
                          'e2': 0.05,
                          'center_x': satellite_centroid_bound/4,
                          'center_y': satellite_centroid_bound/4})
kwargs_lower_lens.append({'theta_E': 0.001,
                          'e1': -0.5,
                          'e2': -0.5,
                          'center_x': satellite_center_x
                                        - satellite_centroid_bound,
                          'center_y': satellite_center_y
                                        - satellite_centroid_bound})
kwargs_upper_lens.append({'theta_E': 2.,
                          'e1': 0.5,
                          'e2': 0.5,
                          'center_x': satellite_center_x
                                        + satellite_centroid_bound,
                          'center_y': satellite_center_y
                                        + satellite_centroid_bound})

## Source Galaxy's Light Model

In [ ]:
# source galaxy's light model
fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []

# setting SERSIC_ELLIPSE parameters
fixed_source.append({"n_sersic": 1.0})
kwargs_source_init.append({"R_sersic": 0.1,
                           "n_sersic": 1.0,
                           "e1": -0.0057,
                           "e2": 0.0736,
                           "center_x": -0.4328,
                           "center_y": 0.0630,
                           "amp": 1,})

kwargs_source_sigma.append({"n_sersic": 0.5,
                            "R_sersic": 0.1,
                            "e1": 0.05,
                            "e2": 0.05,
                            "center_x": 0.2,
                            "center_y": 0.2,
                            "amp": 10,})

kwargs_lower_source.append({"e1": -0.5,
                            "e2": -0.5,
                            "R_sersic": 0.04,
                            "n_sersic": 0.5,
                            "center_x": -10,
                            "center_y": -10,
                            "amp": 0,})

kwargs_upper_source.append({"e1": 0.5,
                            "e2": 0.5,
                            "R_sersic": 0.25,
                            "n_sersic": 5.0,
                            "center_x": 10,
                            "center_y": 10,
                            "amp": 100,})


# setting SHAPELETS parameters
fixed_source.append({"n_max": 10})
kwargs_source_init.append({"beta": 0.0378,
                           "center_x": -0.4328,
                           "center_y": 0.0630})

kwargs_source_sigma.append({"beta": 0.01,
                            "center_x": 0.2,
                            "center_y": 0.2})

kwargs_lower_source.append({"beta": 0.001,
                            "center_x": -10,
                            "center_y": -10})

kwargs_upper_source.append({"beta": 0.15,
                            "center_x": 10,
                            "center_y": 10})

## Lens Galaxy's Light Model

In [ ]:
# lens galaxy's light model
fixed_lens_light = []
kwargs_lens_light_init = []
kwargs_lens_light_sigma = []
kwargs_lower_lens_light = []
kwargs_upper_lens_light = []


# 1st sersic

fixed_lens_light.append({"n_sersic": 1.0})

kwargs_lens_light_init.append({"R_sersic": 0.1400,
                               "n_sersic": 1.0,
                               "e1": 0.0530,
                               "e2": 0.1075,
                               "center_x": -0.1712,
                               "center_y": -0.0184,
                               "amp": 1,})

kwargs_lens_light_sigma.append({"n_sersic": 1,
                                "R_sersic": 0.3,
                                "e1": 0.05,
                                "e2": 0.05,
                                "center_x": 0.1,
                                "center_y": 0.1,
                                "amp": 1,})

kwargs_lower_lens_light.append({"e1": -0.5,
                                "e2": -0.5,
                                "R_sersic": 0.001,
                                "n_sersic": 0.5,
                                "center_x": -10,
                                "center_y": -10,
                                "amp": 0,})

kwargs_upper_lens_light.append({"e1": 0.5,
                                "e2": 0.5,
                                "R_sersic": 10,
                                "n_sersic": 5.0,
                                "center_x": 10,
                                "center_y": 10,
                                "amp": 100,})


# 2nd sersic

fixed_lens_light.append({"n_sersic": 4.0})
kwargs_lens_light_init.append({"R_sersic": 1.0764,
                               "n_sersic": 4.0,
                               "e1": 0.0530,
                               "e2": 0.1075,
                               "center_x": -0.1712,
                               "center_y": -0.0184,
                               "amp": 1,})

kwargs_lens_light_sigma.append({"n_sersic": 1,
                                "R_sersic": 0.3,
                                "e1": 0.05,
                                "e2": 0.05,
                                "center_x": 0.1,
                                "center_y": 0.1,
                                "amp": 10,})

kwargs_lower_lens_light.append({"e1": -0.5,
                                "e2": -0.5,
                                "R_sersic": 0.001,
                                "n_sersic": 0.5,
                                "center_x": -10,
                                "center_y": -10,
                                "amp": 0,})

kwargs_upper_lens_light.append({"e1": 0.5,
                                "e2": 0.5,
                                "R_sersic": 10,
                                "n_sersic": 5.0,
                                "center_x": 10,
                                "center_y": 10,
                                "amp": 100,})


# # third sersic for satellite galaxy
fixed_lens_light.append({'n_sersic': 4.})
kwargs_lens_light_init.append({'R_sersic': 0.2623,
                               'n_sersic': 4,
                               'e1': 0.0056,
                               'e2': 0.0018,
                               'center_x': satellite_center_x,
                               'center_y': satellite_center_y,
                               'amp': 16})
kwargs_lens_light_sigma.append({'n_sersic': 1,
                                'R_sersic': 0.3,
                                'e1': 0.05,
                                'e2': 0.05,
                                'center_x': satellite_centroid_bound/4,
                                'center_y': satellite_centroid_bound/4,
                                'amp': 1})
kwargs_lower_lens_light.append({'e1': -0.5,
                                'e2': -0.5,
                                'R_sersic': 0.001,
                                'n_sersic': .5,
                                'center_x': satellite_center_x
                                            - satellite_centroid_bound,
                                'center_y': satellite_center_y
                                            - satellite_centroid_bound,
                                'amp': 0})
kwargs_upper_lens_light.append({'e1': 0.5,
                                'e2': 0.5,
                                'R_sersic': 10,
                                'n_sersic': 5.,
                                'center_x': satellite_center_x
                                            + satellite_centroid_bound,
                                'center_y': satellite_center_y
                                            + satellite_centroid_bound,
                                'amp': 100})

### Merging source and lens models

In [ ]:
joint_lens_with_lens_light = [[2,2, ["center_x", "center_y", 'e1', 'e2']]]
joint_shapelets_with_sersic = [[0, 1, ["center_x", "center_y"]]]
joint_lens_light_with_lens_light = [[0, 1, ["center_x", "center_y", 'e1', 'e2']]]


lens_params = [
    kwargs_lens_init,
    kwargs_lens_sigma,
    fixed_lens,
    kwargs_lower_lens,
    kwargs_upper_lens,
]

source_params = [
    kwargs_source_init,
    kwargs_source_sigma,
    fixed_source,
    kwargs_lower_source,
    kwargs_upper_source,
]

lens_light_params = [
    kwargs_lens_light_init,
    kwargs_lens_light_sigma,
    fixed_lens_light,
    kwargs_lower_lens_light,
    kwargs_upper_lens_light,
]

## Additional log likelihood

In [ ]:
def custom_log_likelihood_addition(kwargs_lens=None, kwargs_source=None,
                                   kwargs_lens_light=None, kwargs_ps=None,
                                   kwargs_special=None, kwargs_extinction=None):
  from lenstronomy.Util.param_util import ellipticity2phi_q
  import numpy as np
  """

  :param kwargs_lens: lens mass model keywords

  :type kwargs_lens: list

  :param kwargs_source: source light model keywords

  :type kwargs_source: list

  :param kwargs_lens_light: lens light model keywords

  :type kwargs_lens_light:  list

  :param kwargs_ps: point source keywords

  :type kwargs_ps: list

  :param kwargs_special: special keywords

  :type kwargs_special: list

  :param kwargs_extinction: extinction model keywords

  :type kwargs_extinction: list

  :return: custom log_likelihood

  :rtype: float

  """
  # For mass_q, light_q

  log_likelihood_q = 0.

  mass_phi, mass_q = ellipticity2phi_q(kwargs_lens[0]['e1'],
                                       kwargs_lens[0]['e2'])
  
  light_phi, light_q = ellipticity2phi_q(kwargs_lens_light[0]['e1'],
                                       kwargs_lens_light[0]['e2'])

  gaussian_std = 0.003
  if mass_q < light_q:
    log_likelihood_q += -0.5 * (mass_q - light_q)**2 / gaussian_std**2



  # For theta_E_lens0 / theta_E_lens2
  
  log_likelihood_mass = 0.0

  central_light = 2614.0328912829286   # total flux of central lens
  satellite_light = 47.81843573926484   # total flux of satellite lens
  uncertainty_central = 0.1 * central_light
  uncertainty_satellite = 0.1 * satellite_light

  ratio_expected = np.sqrt(satellite_light / central_light)
  gaussian_std_mass = (
      0.5
      * np.sqrt(
          (uncertainty_central / central_light) ** 2
          + (uncertainty_satellite / satellite_light) ** 2
      )
      * ratio_expected
  )

  ratio = kwargs_lens[2]["theta_E"] / kwargs_lens[0]["theta_E"]

  log_likelihood_mass += -0.5 * (ratio - ratio_expected) ** 2 / gaussian_std_mass**2

  return log_likelihood_q + log_likelihood_mass

### Combining all the above specification in the `kwargs_params` dictionary

In [ ]:
kwargs_params = { "lens_model": lens_params,
                 "source_model": source_params,
                 "lens_light_model": lens_light_params}
kwargs_constraints = {
    "joint_lens_with_light" : joint_lens_with_lens_light,
    "joint_source_with_source": joint_shapelets_with_sersic,
    "joint_lens_light_with_lens_light": joint_lens_light_with_lens_light
}

kwargs_likelihood = {'custom_logL_addition': custom_log_likelihood_addition,
                     "check_bounds": True, "image_likelihood_mask_list": [mask]}

kwargs_numerics = {"supersampling_factor": 3, "supersampling_convolution": False}

### Compiling all the information that needs to be transmitted to Lenstronomy

In [ ]:
kwargs_model = {
    "lens_model_list": lens_model_list,
    "source_light_model_list": source_model_list,
    "lens_light_model_list": lens_light_model_list,
}

multi_band_list = [[kwargs_data, kwargs_psf, kwargs_numerics]]

kwargs_data_joint = {
    "multi_band_list": multi_band_list,
    "multi_band_type": "single-band",
}

 ### Model fiting

In [ ]:
fitting_seq = FittingSequence(
    kwargs_data_joint,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params
)

fitting_kwargs_list = [
    ["PSO", {"sigma_scale": 1.0, "n_particles": 100, "n_iterations": 10}]
]

chain_list = fitting_seq.fit_sequence(fitting_kwargs_list)
kwargs_result = fitting_seq.best_fit()

In [ ]:
multi_band_list_out = fitting_seq.multi_band_list
kwargs_fixed_out = fitting_seq.kwargs_fixed

init_samples = None  # can be not `None` for MCMC

input = [
    fitting_kwargs_list,
    multi_band_list,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
    init_samples,
]
output = [kwargs_result, multi_band_list_out, chain_list, kwargs_fixed_out]

output_path = "./DESIJ0618+5018_pso_output_V.joblib"

with open(output_path, "wb") as f:
    joblib.dump([input, output], f, compress=True)

In [ ]:
### load saved best fit parameters ###

load_output_path = "DESIJ0618+5018_pso_output_V.joblib"
with open(load_output_path, "rb") as f:
    [input_, output_] = joblib.load(f)

(
    fitting_kwargs_list,
    multi_band_list,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
    init_samples,
) = input_

kwargs_result, multi_band_list_out, fit_output, _ = output_

### Illustrating the fitted model

In [ ]:
model_plot = ModelPlot(
    multi_band_list,
    kwargs_model,
    kwargs_result,
    arrow_size=0.02,
    cmap_string="cubehelix",
    image_likelihood_mask_list=kwargs_likelihood["image_likelihood_mask_list"],
)

f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

model_plot.data_plot(ax=axes[0, 0])
model_plot.model_plot(ax=axes[0, 1])
model_plot.normalized_residual_plot(ax=axes[0, 2], v_min=-3, v_max=3, cmap="RdBu_r")
model_plot.source_plot(
    ax=axes[1, 0], deltaPix_source=0.05, with_caustics=True, scale_size=0.5, numPix=100
)
model_plot.convergence_plot(ax=axes[1, 1], v_max=1, cmap="gist_heat")
model_plot.magnification_plot(ax=axes[1, 2], cmap="PiYG")
f.tight_layout()
f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.0, hspace=0.05)
plt.show()

f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

model_plot.decomposition_plot(
    ax=axes[0, 0], text="Lens light", lens_light_add=True, unconvolved=True
)
model_plot.decomposition_plot(
    ax=axes[1, 0], text="Lens light convolved", lens_light_add=True
)
model_plot.decomposition_plot(
    ax=axes[0, 1], text="Source light", source_add=True, unconvolved=True
)
model_plot.decomposition_plot(
    ax=axes[1, 1], text="Source light convolved", source_add=True
)
model_plot.decomposition_plot(
    ax=axes[0, 2],
    text="All components",
    source_add=True,
    lens_light_add=True,
    unconvolved=True,
)
model_plot.decomposition_plot(
    ax=axes[1, 2],
    text="All components convolved",
    source_add=True,
    lens_light_add=True,
    point_source_add=True,
)
f.tight_layout()
f.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.0, hspace=0.05)
plt.show()
print(kwargs_result)

In [ ]:
result_mass_e1 = kwargs_result['kwargs_lens'][0]['e1']
result_mass_e2 = kwargs_result['kwargs_lens'][0]['e2']

result_light_e1 = kwargs_result['kwargs_lens_light'][0]['e1']
result_light_e2 = kwargs_result['kwargs_lens_light'][0]['e2']

mass_phi, mass_q = ellipticity2phi_q(result_mass_e1, result_mass_e2)
light_phi, light_q = ellipticity2phi_q(result_light_e1, result_light_e2)

print("mass_q: {},\nlight_q: {}".format(mass_q, light_q,))

In [ ]:
lens_central_model_list = [lens_light_model_list[0], lens_light_model_list[1]]
lens_satellite_model_list = [lens_light_model_list[2]]

light_model_central = LightModel(lens_central_model_list)
analysis_central = LightProfileAnalysis(light_model_central)

light_model_satellite = LightModel(lens_satellite_model_list)
analysis_satellite = LightProfileAnalysis(light_model_satellite)

# Providing the lens light results from PSO
kwargs_central_light = [
    kwargs_result["kwargs_lens_light"][0],
    kwargs_result["kwargs_lens_light"][1],
]
kwargs_satellite_light = [kwargs_result["kwargs_lens_light"][2]]


# Calculate the total flux of central lens
total_flux_central = analysis_central.flux_components(
    kwargs_central_light, grid_spacing=0.01, grid_num=10 / 0.01
)
# Calculate the total flux of satellite
total_flux_satellite = analysis_satellite.flux_components(
    kwargs_satellite_light, grid_spacing=0.01, grid_num=10 / 0.01
)


print("Total Central lens flux:", np.sum(total_flux_central))
print("Total satellite flux:", np.sum(total_flux_satellite))

In [ ]:
mcmc_backend = "./mcmc_backend.h5"

n_walkers = 100
n_step = 10
n_burn = 0

fitting_kwargs_list_mcmc = [
    # ["update_settings", {"lens_remove_fixed": [[0, ["gamma"]]]}],
    ['MCMC', {'n_burn': n_burn,
                                      'n_run': n_step,
                                      'n_walkers': n_walkers,
                                      'sigma_scale': 0.1,
                                      'threadCount': 6,
                                      'backend_filename': mcmc_backend,
                                      'start_from_backend': False}]]

chain_list_mcmc = fitting_seq.fit_sequence(fitting_kwargs_list_mcmc)
kwargs_result_mcmc = fitting_seq.best_fit()

### Trace Plot

In [ ]:
samples_mcmc = []

if chain_list_mcmc[-1][0] != "PSO":
    # if MCMC chain was broken in the chunks,
    # we join the chunks to make the full chain
    mcmc_repeat = 1

    for k in range(len(chain_list_mcmc) - mcmc_repeat, len(chain_list_mcmc)):
        if samples_mcmc == []:
            samples_mcmc = chain_list_mcmc[k][1]
        else:
            samples_mcmc = np.vstack((samples_mcmc, chain_list_mcmc[k][1]))

        param_mcmc = chain_list_mcmc[k][2]

if not samples_mcmc == []:
    n_params = samples_mcmc.shape[1]
    n_walkers = 100
    n_step = 1500
    n_burn = 0

    print("N_step: {}, N_walkers: {}, N_params: {}".format(n_step, n_walkers,
                                                           n_params))

    chain = np.empty((n_walkers, n_step, n_params))

    for i in range(n_params):
        samples = samples_mcmc[:, i]
        reshaped_samples = samples.reshape((n_walkers, n_step), order="F")
        chain[:, :, i] = reshaped_samples
    mean_pos = np.zeros((n_params, n_step))
    median_pos = np.zeros((n_params, n_step))
    std_pos = np.zeros((n_params, n_step))
    q16_pos = np.zeros((n_params, n_step))
    q84_pos = np.zeros((n_params, n_step))

    for i in range(n_params):
        for j in range(n_step):
            mean_pos[i][j] = np.mean(chain[:, j, i])
            median_pos[i][j] = np.median(chain[:, j, i])
            std_pos[i][j] = np.std(chain[:, j, i])
            q16_pos[i][j] = np.percentile(chain[:, j, i], 16.0)
            q84_pos[i][j] = np.percentile(chain[:, j, i], 84.0)

    fig, ax = plt.subplots(n_params, sharex=True, figsize=(8, 6))

    burnin = -1
    last = n_step

    medians = []
    param_values = [
        median_pos[0][last - 1],
        (q84_pos[0][last - 1] - q16_pos[0][last - 1]) / 2,
        median_pos[1][last - 1],
        (q84_pos[1][last - 1] - q16_pos[1][last - 1]) / 2,
    ]

    for i in range(n_params):
        print(
            param_mcmc[i],
            "{:.4f} ± {:.4f}".format(
                median_pos[i][last - 1],
                (q84_pos[i][last - 1] - q16_pos[i][last - 1]) / 2,
            ),
        )

        ax[i].plot(median_pos[i][:last], c="g")
        ax[i].axhline(np.median(median_pos[i][burnin:last]), c="r", lw=1)
        ax[i].fill_between(
            np.arange(last), q84_pos[i][:last], q16_pos[i][:last], alpha=0.4
        )

        ax[i].set_ylabel(param_mcmc[i], fontsize=10)
        ax[i].set_xlim(0, last)

        medians.append(np.median(median_pos[i][burnin:last]))
    if True:
        fig.set_size_inches((12.0, 2 * len(param_mcmc)))
        """
        plt.savefig("/Trace_Plot_DESIJ1018-0121.png",
                    format="png",
                    bbox_inches="tight")
        """
        plt.show()

### Visualization of MCMC Fit

In [ ]:
model_plot = ModelPlot(
    multi_band_list, kwargs_model, kwargs_result_mcmc,
    arrow_size=0.02, cmap_string="cubehelix",
    image_likelihood_mask_list=kwargs_likelihood['image_likelihood_mask_list']
)

f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

model_plot.data_plot(ax=axes[0,0])
model_plot.model_plot(ax=axes[0,1])
model_plot.normalized_residual_plot(ax=axes[0,2], v_min=-3, v_max=3,
                                   cmap='RdBu_r')
model_plot.source_plot(ax=axes[1, 0], deltaPix_source=0.03, with_caustics=True,
                        scale_size=0.5, numPix=100)
model_plot.convergence_plot(ax=axes[1, 1], v_max=1, cmap='gist_heat')
model_plot.magnification_plot(ax=axes[1, 2], cmap='PiYG')
f.tight_layout()
f.subplots_adjust(left=None, bottom=None, right=None, top=None,
                  wspace=0., hspace=0.05)
plt.show()

f, axes = plt.subplots(2, 3, figsize=(16, 8), sharex=False, sharey=False)

model_plot.decomposition_plot(ax=axes[0,0], text='Lens light',
                             lens_light_add=True, unconvolved=True)
model_plot.decomposition_plot(ax=axes[1,0], text='Lens light convolved',
                             lens_light_add=True)
model_plot.decomposition_plot(ax=axes[0,1], text='Source light',
                             source_add=True, unconvolved=True)
model_plot.decomposition_plot(ax=axes[1,1], text='Source light convolved',
                             source_add=True)
model_plot.decomposition_plot(ax=axes[0,2], text='All components',
                             source_add=True, lens_light_add=True,
                             unconvolved=True)
model_plot.decomposition_plot(ax=axes[1,2], text='All components convolved',
                             source_add=True, lens_light_add=True,
                             point_source_add=True)
f.tight_layout()
f.subplots_adjust(left=None, bottom=None, right=None, top=None,
                  wspace=0., hspace=0.05)
plt.show()
print(kwargs_result_mcmc)

### Corner Plot

In [ ]:
if len(chain_list_mcmc) > 0:
    sampler_type, samples_mcmc, param_mcmc, dist_mcmc = chain_list_mcmc[0]

    param_class = fitting_seq.param_class

    print("Number of non-linear parameters in the MCMC process:", len(param_mcmc))
    print("Parameters in order:", param_mcmc)
    if samples_mcmc is not None:
        print("Number of evaluations in the MCMC process:", np.shape(samples_mcmc)[0])
        n_sample = len(samples_mcmc)
        print(n_sample)
        burnin = 1200
        thin = 5
        samples_mcmc_cut = chain[:, burnin::thin, :].reshape((-1, n_params))
        if not samples_mcmc_cut == []:
            n, num_param = np.shape(samples_mcmc_cut)
            print("Shape of samples_mcmc_cut:", samples_mcmc_cut.shape)
            plot = corner.corner(samples_mcmc_cut, labels=param_mcmc, show_titles=True)
    else:
        print("No samples available for corner plot.")
else:
    print("No MCMC chains available.")

In [ ]:
lens_central_model_list = [lens_light_model_list[0], lens_light_model_list[1]]
lens_satellite_model_list = [lens_light_model_list[2]]

light_model_central = LightModel(lens_central_model_list)
analysis_central = LightProfileAnalysis(light_model_central)

light_model_satellite = LightModel(lens_satellite_model_list)
analysis_satellite = LightProfileAnalysis(light_model_satellite)

# Providing the lens light results from PSO
kwargs_central_light = [
    kwargs_result["kwargs_lens_light"][0],
    kwargs_result["kwargs_lens_light"][1],
]
kwargs_satellite_light = [kwargs_result["kwargs_lens_light"][2]]


# Calculate the total flux of central lens
total_flux_central = analysis_central.flux_components(
    kwargs_central_light, grid_spacing=0.01, grid_num=10 / 0.01
)
# Calculate the total flux of satellite
total_flux_satellite = analysis_satellite.flux_components(
    kwargs_satellite_light, grid_spacing=0.01, grid_num=10 / 0.01
)


print("Total Central lens flux:", np.sum(total_flux_central))
print("Total satellite flux:", np.sum(total_flux_satellite))

In [ ]:
result_mass_e1 = kwargs_result_mcmc['kwargs_lens'][0]['e1']
result_mass_e2 = kwargs_result_mcmc['kwargs_lens'][0]['e2']

result_light_e1 = kwargs_result_mcmc['kwargs_lens_light'][0]['e1']
result_light_e2 = kwargs_result_mcmc['kwargs_lens_light'][0]['e2']

mass_phi, mass_q = ellipticity2phi_q(result_mass_e1, result_mass_e2)
light_phi, light_q = ellipticity2phi_q(result_light_e1, result_light_e2)

print("mass_q: {},\nlight_q: {}".format(mass_q, light_q,))

### MCMC fit output

In [ ]:

multi_band_list_out = fitting_seq.multi_band_list
kwargs_fixed_out = fitting_seq.kwargs_fixed

init_samples = 5000  # cannot be 'None' for MCMC

input = [
    fitting_kwargs_list_mcmc,
    multi_band_list,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
    init_samples,
]

output = [kwargs_result_mcmc, multi_band_list_out, chain_list_mcmc, kwargs_fixed_out]

output_path = "mcmc_output.joblib"

with open(output_path, "wb") as f:
    joblib.dump([input, output], f, compress=True)

In [ ]:
multi_band_list_out = fitting_seq.multi_band_list
kwargs_fixed_out = fitting_seq.kwargs_fixed

init_samples = 5000  # cannot be 'None' for MCMC

input = [
    fitting_kwargs_list_mcmc,
    multi_band_list,
    kwargs_model,
    kwargs_constraints,
    kwargs_likelihood,
    kwargs_params,
    init_samples,
]

output = [kwargs_result_mcmc, multi_band_list_out, kwargs_fixed_out]

output_path = "mcmc_output_without_chain_list.joblib"

with open(output_path, "wb") as f:
    joblib.dump([input, output], f, compress=True)